In [142]:
import pandas as pd
import os
import base64
import json
import requests
from io import StringIO

In [ ]:
username = 'brendancapple'
repository_name = 'FloraFrontier'


In [144]:
def github_read_file(username, repository_name, file_path, github_token=None):
    headers = {}
    if github_token:
        headers['Authorization'] = f"token {github_token}"
        
    url = f'https://api.github.com/repos/{username}/{repository_name}/contents/{file_path}'
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json() 
    file_content = data['content']
    file_content_encoding = data.get('encoding')
    if file_content_encoding == 'base64':
        file_content = base64.b64decode(file_content).decode()

    return file_content

def load_distributionData(scientific):
    content = github_read_file(username, repository_name, "data/DistributionData_" + scientific + ".csv", github_token)
    output = pd.read_csv(StringIO(content), sep=',', header=1)
    return output

def load_searchData(state):
    content = github_read_file(username, repository_name, "data/SearchResults_" + state + ".csv", github_token)
    output = pd.read_csv(StringIO(content), sep=',', header=1)
    return output

In [145]:
load_list = ['triticumAestivum', 'brassicaRapa', 'glycineMax', 'lactucaSativa', 'solanumLycopersicum', 'solanumTuberosum']
distribution_list = []

for plant in load_list:
    distribution_list.append(load_distributionData(plant))
    print(plant)

# load_distributionData("triticumAestivum")

# distribution_list[0]
len(distribution_list)

triticumAestivum
brassicaRapa
glycineMax
lactucaSativa
solanumLycopersicum
solanumTuberosum


6

In [146]:
results_dictionary = dict()
state_results_dictionary = dict()

agreed_counties = set()
agreed_states = set()

for distribution in distribution_list:
    counties_present = set()
    states_present = set()

    print("---NEW DISTRIBUTION---")
    # distribution.apply(dict_handling)
    for index, row in distribution.iterrows():
        # print(row['County'], row['State'] , row['Country'])
        key = str(row['County']) + " - " + str(row['State']) + " - " + str(row['Country'])
        state_key = str(row['State'])

        counties_present.add(key)
        states_present.add(state_key)

        print(key)
        print(state_key)
        if key in results_dictionary:
            results_dictionary[key] = results_dictionary[key] + 1
            # print(">> key++")
        else:
            results_dictionary[key] = 1
            # print(">> key=1")

        if state_key in state_results_dictionary:
            state_results_dictionary[state_key] = state_results_dictionary[state_key] + 1
            # print(">> state_key++")
        else:
            state_results_dictionary[state_key] = 1
            # print(">> state_key=1")

    if len(agreed_counties) == 0:
        agreed_counties = counties_present
        agreed_states = states_present
    else:
        temp_counties = set()
        temp_states = set()
        for county in agreed_counties:
            if county in counties_present:
                temp_counties.add(county)
        agreed_counties = counties_present

        for state in agreed_states:
            if state in states_present:
                temp_states.add(state)
        agreed_states = temp_states



---NEW DISTRIBUTION---
nan - Alberta - Canada
Alberta
nan - Alabama - United States
Alabama
Baldwin - Alabama - United States
Alabama
Conecuh - Alabama - United States
Alabama
Lee - Alabama - United States
Alabama
Limestone - Alabama - United States
Alabama
Marshall - Alabama - United States
Alabama
Mobile - Alabama - United States
Alabama
Montgomery - Alabama - United States
Alabama
Pickens - Alabama - United States
Alabama
Tuscaloosa - Alabama - United States
Alabama
nan - British Columbia - Canada
British Columbia
nan - Alaska - United States
Alaska
Anchorage - Alaska - United States
Alaska
Kodiak Island - Alaska - United States
Alaska
nan - Manitoba - Canada
Manitoba
nan - New Brunswick - Canada
New Brunswick
nan - Arizona - United States
Arizona
Apache - Arizona - United States
Arizona
Coconino - Arizona - United States
Arizona
Maricopa - Arizona - United States
Arizona
Pima - Arizona - United States
Arizona
Santa Cruz - Arizona - United States
Arizona
nan - Arkansas - United Stat

Beauregard - Louisiana - United States
Louisiana
Bossier - Louisiana - United States
Louisiana
Caddo - Louisiana - United States
Louisiana
Calcasieu - Louisiana - United States
Louisiana
Caldwell - Louisiana - United States
Louisiana
De Soto - Louisiana - United States
Louisiana
East Baton Rouge - Louisiana - United States
Louisiana
East Carroll - Louisiana - United States
Louisiana
East Feliciana - Louisiana - United States
Louisiana
Franklin - Louisiana - United States
Louisiana
Iberia - Louisiana - United States
Louisiana
Jackson - Louisiana - United States
Louisiana
Jefferson - Louisiana - United States
Louisiana
Jefferson Davis - Louisiana - United States
Louisiana
Lafayette - Louisiana - United States
Louisiana
Lincoln - Louisiana - United States
Louisiana
Livingston - Louisiana - United States
Louisiana
Madison - Louisiana - United States
Louisiana
Morehouse - Louisiana - United States
Louisiana
Natchitoches - Louisiana - United States
Louisiana
Orleans - Louisiana - United Stat

In [147]:
total_locations = sum(results_dictionary.values())
total_locations

1865

In [148]:
agreed_states

{'Alabama',
 'Illinois',
 'Massachusetts',
 'Michigan',
 'Missouri',
 'New York',
 'Ohio',
 'Ontario',
 'Pennsylvania'}

In [149]:
combined_distribution = pd.DataFrame(results_dictionary.items(), columns = ['County', 'Frequency'])
combined_distribution = combined_distribution.sort_values(by='Frequency', ascending=False)

state_combined_distribution = pd.DataFrame(state_results_dictionary.items(), columns = ['State', 'Frequency'])
state_combined_distribution = state_combined_distribution.sort_values(by='Frequency', ascending=False)

In [150]:
combined_distribution.head(20)

,County,Frequency
238,Berkshire - Massachusetts - United States,6
112,nan - Illinois - United States,6
357,nan - New York - United States,6
288,nan - Missouri - United States,6
445,Northampton - Pennsylvania - United States,6
1,nan - Alabama - United States,6
443,Montgomery - Pennsylvania - United States,6
403,nan - Ohio - United States,6
248,nan - Michigan - United States,6
440,Lehigh - Pennsylvania - United States,6


In [151]:
state_combined_distribution.head(20)

,State,Frequency
24,Illinois,151
29,Louisiana,146
48,Pennsylvania,106
8,California,91
43,New York,80
56,Virginia,79
52,Tennessee,74
45,Ohio,73
33,Michigan,63
36,Missouri,53


In [152]:
state_search_results = []

for state in list(agreed_states):
    print(state)
    state_results = load_searchData(state)

    if len(state_search_results) == 0:
        state_search_results = state_results['Scientific Name']

Alabama


HTTPError: 404 Client Error: Not Found for url: https://api.github.com/repos/brendancapple/FloraFrontier/contents/data/SearchResults_Alabama.csv